In [10]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import pycountry_convert as pc

%matplotlib inline

In [11]:
df = pd.read_csv('https://raw.githubusercontent.com/rfordatascience/tidytuesday/master/data/2020/2020-12-22/big-mac.csv')

In [12]:
df['date'] = pd.to_datetime(df['date'],format='%Y-%m-%d')

In [13]:
df = df.dropna()

In [23]:
continent_dict = {
    'EU':'Europe',
    'SA':'South America',
    'OC':'Oceania',
    'NA':'North America',
    'AF':'Africa',
    'AS':'Asia'    
}

In [27]:
def perc_null(df):
    
    cols = df.columns
    row_format = '{:20}''{:10}'
    
    for col in cols: 
        
        ratio = df[col].isnull().sum()/df.shape[0]
        print(row_format.format(col,format(ratio,'.2%')))   
        
def create_continent_column(iso_a3):
    
    if iso_a3 == 'EUZ':
        return 'EU'
    else:
        iso_a2 = pc.country_alpha3_to_country_alpha2(iso_a3)
        return pc.country_alpha2_to_continent_code(iso_a2)

In [28]:
df['Continent'] = df['iso_a3'].apply(create_continent_column)
df['Continent'] = df['Continent'].map(continent_dict)

In [153]:
def plot_currency_imbalance_year(df,ref_currency,year):
    
    curr_col = ref_currency + '_adjusted'    
    df_year = df[(df['date'].dt.year==year)&(df['date'].dt.month>6)]
    
    plt.scatter(df_year[df_year[curr_col]>0]['dollar_ex'],
                df_year[df_year[curr_col]>0][curr_col],
                marker='o',
                s=100*(df_year[df_year[curr_col]>0]['gdp_dollar']/\
                      df[df['name']=='United States']['gdp_dollar'].iloc[-1]),
                alpha=0.6,
                color='green')
    plt.scatter(df_year[df_year[curr_col]<0]['dollar_ex'],
                df_year[df_year[curr_col]<0][curr_col],
                marker='o',
                s=100*(df_year[df_year[curr_col]<0]['gdp_dollar']/\
                      df[df['name']=='United States']['gdp_dollar'].iloc[-1]),
                alpha=0.6,
                color='red')
    plt.xlim([0,100])
    
    
def plot_currency_imbalance_year_plotly(df,ref_currency,year):
    
    curr_col = ref_currency + '_adjusted'    
    df_year = df[(df['date'].dt.year==year)&(df['date'].dt.month>6)]
    fig = px.scatter(df_year,
                     x='dollar_ex',
                     y= 100*df_year[curr_col],
                     size='gdp_dollar',               
                     hover_name='name',
                     hover_data={'name':False,
                                 'Continent':False,
                                 'dollar_ex':False,
                                 'gdp_dollar':':.0f'
                                 
                                },
                     text='name',
                     color='Continent',
                     log_x=True,
                     size_max=60,                
)

    fig.update_xaxes(title_text='Local currency units per dollar',
                     title_font={'size':18},
                     tickvals=[1,10,100,1000,10000],
                     tickfont=dict(size=16))
    fig.update_yaxes(title_text='Adjusted index, relative to {}'.format(ref_currency.upper()+' (%)'),
                     title_font={'size':18},
                     tickfont=dict(size=16))
    fig.update_layout(
        height=600,
        width=900,        
        title_text='Burger price difference from '+ref_currency.upper()+' %, '+str(year),
        title_font={'size':24},
        hoverlabel=dict(bgcolor='white')
    )
    
    
    fig.show()
    
    
def plot_currency_imbalance_country_plotly(df,ref_currency,countries):
    
    curr_col = ref_currency + '_adjusted'    
    df_country = df[df['name'].isin(countries)]
    fig = px.line(df_country,
                  x='date',
                  y= 100*df_country[curr_col],                      
                  color='name',
                  hover_data={'name':False}
)

    fig.update_traces(mode="markers+lines",
                     hovertemplate='Burger price offset: %{y}%<extra></extra>')
    fig.update_xaxes(title_text='Year',
                     title_font={'size':18},
                     tickfont=dict(size=16))
    fig.update_yaxes(title_text='Adjusted index, relative to {}'.format(ref_currency.upper()+' (%)'),
                     title_font={'size':18},
                     tickfont=dict(size=16))
    fig.update_layout(
        height=600,
        width=900,        
        title_text='Burger price difference from '+ref_currency.upper()+' %',
        title_font={'size':24},
        hoverlabel=dict(
            bgcolor='white'
        )
    )
    
    
    fig.show()
    
    
def plot_currency_imbalance_country(df,ref_currency,country):
    
    curr_col = ref_currency + '_adjusted'    
    df_country = df[df['name']==country]
    plt.scatter(df_country[df_country[curr_col]>0]['date'],
                df_country[df_country[curr_col]>0][curr_col]*100,
                marker='o',
                s=100*(df_country[df_country[curr_col]>0]['gdp_dollar']/\
                      df[df['name']=='United States']['gdp_dollar'].iloc[-1]),
                color='green',
                alpha=0.6
               )
    plt.scatter(df_country[df_country[curr_col]<0]['date'],
                df_country[df_country[curr_col]<0][curr_col]*100,
                marker='o',
                s=100*(df_country[df_country[curr_col]<0]['gdp_dollar']/\
                      df[df['name']=='United States']['gdp_dollar'].iloc[-1]),
                color='red',
                alpha=0.6
               )
    plt.title('Burger price imbalance: {}'.format(country),fontsize=16)
    plt.xlabel('Year',fontsize=14)
    plt.ylabel('Price difference from '+ref_currency.upper()+' %',fontsize=14)

In [154]:
plot_currency_imbalance_year_plotly(df,'cny',2011)

In [155]:
countries = ['Argentina','Brazil','Britain','Japan','Hong Kong','Switzerland']

In [156]:
plot_currency_imbalance_country_plotly(df,'cny',countries)